# Web Scraping and time series

In [2]:
#import the necessary packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
import regex as re
import matplotlib as plt
from bs4 import BeautifulSoup as bsoup
%matplotlib inline

### go to the Eurosport Paris Roubaix cycling results page and load it into a pandas dataframe

https://www.eurosport.com/cycling/paris-roubaix/2022/live-road-race-men_mtc1323920/live.shtml

Hint, pandas read_html() will fail to find the table even though it is on the page.  You need to find the table (class='standing-table') with a selenium driver.get() or BeautifulSoup, then pass that to pandas read_html().

In [111]:
# Reading in the web page Then displaying

# Eurosport Paris Roabix cycling results
url = "https://www.eurosport.com/cycling/paris-roubaix/2022/live-road-race-men_mtc1323920/live.shtml"

# Make the request and open the url :
# req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
response = requests.get(url)
html = response.content

soup = bsoup(html, 'html.parser')
soup.find_all("tr")

# //*[@id="col-left"]/div/div[5]/div/div/div/table/tbody/tr[1]
# webpage = urlopen(req).read()

# # Create a beutiful soup object so we can work with the data
# soup = bsoup(webpage, 'html.parser')
# soup.find_all("standing-table__row")

#Sanity check 
# clean = soup.body.prettify()
# for line in clean:
#     print(line)
#     if line == "class='standing-table'":
#         print(line)


[]

In [69]:
for link in soup.find_all("a"):
    if link.text == 
    
    print("Inner Text: {}".format(link.text))

    print("Title: {}".format(link.get("title")))
    print("href: {}".format(link.get("href")))

# tables = soup.body.find_all("div")
# for table in tables:
#     print("New Line \n")
#     print(table)
#     print("New Line Coming in HOt \n")




# table = soup.find_all('div', attrs={"class":"standing-table"})
# mydivs = soup.find_all('table')
# mydivs2 = soup.find_all('table', attrs={"class":"standing-table"})
# mydivs2



# len(mydivs)
# for div in mydivs:
#     print(div)
#     print("End fo Div \n")

# for div in mydivs:
#     print("New Line =>", div)


# # Let's find the data from the table :
# table = soup.find('table', attrs={'class': 'tableSorter2'})
# # table
# results = table.find_all('tr')
# print('Number of results', len(results))
# print("Table", table)


Inner Text: 
Title: None
href: https://www.eurosport.com
Inner Text: Video
Title: None
href: https://video.eurosport.com
Inner Text: Results
Title: None
href: /score-center.shtml
Inner Text: Football
Title: None
href: /football/
Inner Text: Cycling
Title: None
href: /cycling/
Inner Text: Snooker
Title: None
href: /snooker/
Inner Text: Football
Title: None
href: /football/
Inner Text: Cycling
Title: None
href: /cycling/
Inner Text: Snooker
Title: None
href: /snooker/
Inner Text: Tennis
Title: None
href: /tennis/
Inner Text: Motorsports
Title: None
href: /allmotorsports/
Inner Text: Winter Sports
Title: None
href: /winter-sports/
Inner Text: Athletics
Title: None
href: /athletics/
Inner Text: Athletics
Title: None
href: /athletics/
Inner Text: BSB
Title: None
href: /british-superbikes/
Inner Text: Cricket
Title: None
href: /cricket/
Inner Text: Cycling
Title: None
href: /cycling/
Inner Text: Cycling - Track
Title: None
href: /cycling-track/
Inner Text: Darts
Title: None
href: /darts/
Inn

### For the next part of the homework you will be scraping data from the following URL

# US FDA web page
https://www.accessdata.fda.gov/scripts/cder/daf/

You are going to go to the FDA web page and search for Fentanyl.  It should return a list of all drugs that contain fentanyl.  The table that comes back is just a simple list of the drug names.  If you click on the drug name, it expands the row to show Drug Name, Link, NDA number, Dosage Form, Marketing Status, and Manufacturer.  You need to scrape this web page and print those values seperated by commas.

### POST forms
POST forms do not put the data in the URL, the data is submitted via javascript.  You have to pretend to fill out the form and submit it.  Grabbing the parameters
First we need to find out what parameters we’re going to hunt down. To do this, first make your way to the form, then get prepared.

1) In Chrome, more tools > Developer Tools 2) Click the Network tab 3) Fill the form out, and submit it 4) Scroll up to the top of the Network pane, select the segment of the URL you’re at (I’m at tempai.cfm) 5) Click it 6) Select Headers on the right 7) Scroll down until you see Form Data

### 1)  Part 1, enter the search term fentanyl and submit

In [161]:


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Placed the Driver in the HW File
DRIVER_PATH = 'chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Labeling and getting the Website
url = 'https://www.accessdata.fda.gov/scripts/cder/daf/'
driver.get(url)

# Quering for Fentanyl and submitting
driver.find_element(By.NAME, "searchterm").send_keys("fentanyl" + Keys.ENTER)


### 2) Pull the data from the website, check the status code to make sure it was successfull and create a BeautifulSoup object and parse the data.  Print the soup object below:

In [181]:
pageSource = driver.page_source
soup = bsoup(pageSource, 'html.parser')

data_dict = {}
# Grabbing all the Results
results = soup.find_all('tr')
for result in results:
    inner = result.find_all('li')
    print(inner[0].get_text())
    # print(result.findChildren('li'))
    print("New Line\n")

ABSTRAL (FENTANYL CITRATE) | NDA   #022510 | TABLET;SUBLINGUAL | Discontinued | SENTYNL THERAPS INC
    
New Line

ACTIQ (FENTANYL CITRATE) | NDA   #020747 | TROCHE/LOZENGE;TRANSMUCOSAL | Prescription | CEPHALON
    
New Line

DURAGESIC-100 (FENTANYL) | NDA   #019813 | FILM, EXTENDED RELEASE;TRANSDERMAL | Discontinued | JANSSEN PHARMS
    
New Line

DURAGESIC-12 (FENTANYL) | NDA   #019813 | FILM, EXTENDED RELEASE;TRANSDERMAL | Discontinued | JANSSEN PHARMS
    
New Line

DURAGESIC-25 (FENTANYL) | NDA   #019813 | FILM, EXTENDED RELEASE;TRANSDERMAL | Discontinued | JANSSEN PHARMS
    
New Line

DURAGESIC-37 (FENTANYL) | NDA   #019813 | FILM, EXTENDED RELEASE;TRANSDERMAL | Discontinued | JANSSEN PHARMS
    
New Line

DURAGESIC-50 (FENTANYL) | NDA   #019813 | FILM, EXTENDED RELEASE;TRANSDERMAL | Discontinued | JANSSEN PHARMS
    
New Line

DURAGESIC-75 (FENTANYL) | NDA   #019813 | FILM, EXTENDED RELEASE;TRANSDERMAL | Discontinued | JANSSEN PHARMS
    
New Line

FENTANYL (FENTANYL CITRATE) 

### 3) import selenium webdriver and BeautifulSoup

Get your webdriver and get the page.  Submit the search term and click the search button.

In [ ]:
# I believe I have done this above

### 4) run the selenium page_source through beautiful soup using the html parser

In [125]:
# About to go through the page
# Grabbing all the Results


[<div class="content-block-item result" id="result-1">
 <h4 class="title">
 <a data-click='{"position":1,"module_code":"I14Y"}' href="https://www.fda.gov/drugs/information-drug-class/transmucosal-immediate-release-fentanyl-tirf-medicines">Transmucosal Immediate-Release <strong>Fentanyl</strong> (TIRF) Medicines | FDA</a>
 </h4>
 <span class="url">https://www.fda.gov/drugs/information-drug-class/transmucosal-immediate-release-fentanyl-tirf-medicines</span>
 <span class="description">
 
 ...immediate-release <strong>fentanyl</strong> (TIRF) medicines contain <strong>fentanyl</strong>, a powerful prescription...Transmucosal Immediate-Release <strong>Fentanyl</strong> (TIRF) Medicines Share ...
 </span>
 </div>,
 <div class="content-block-item result" id="result-2">
 <h4 class="title">
 <a data-click='{"position":2,"module_code":"I14Y"}' href="https://www.fda.gov/drugs/postmarket-drug-safety-information-patients-and-providers/index-drug-specific-information">Index to Drug-Specific Informat

### 5) Grab all the rows from the table using find_all.  Note that some of the data is in a 'tr' header and some is in 'li'.

Parse the data and print the results

Note the link on the page is a relative link.  You must join that link with 'https://www.accessdata.fda.gov' to make it a link you can click on.
Example:  https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=022510

### The output should look like this (except print all rows)
Drug Name,Link,NDA number,Dosage Form,Marketing Status,Manufacturer

ABSTRAL , https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=022510 , NDA   #022510 , TABLET;SUBLINGUAL , Discontinued , SENTYNL THERAPS INC

ACTIQ , https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=020747 , NDA   #020747 , TROCHE/LOZENGE;TRANSMUCOSAL , Prescription , CEPHALON

DURAGESIC-100 , https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=019813 , NDA   #019813 , FILM, EXTENDED RELEASE;TRANSDERMAL , Discontinued , JANSSEN PHARMS

DURAGESIC-12 , https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=019813 , NDA   #019813 , FILM, EXTENDED RELEASE;TRANSDERMAL , Discontinued , JANSSEN PHARMS

DURAGESIC-25 , https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=019813 , NDA   #019813 , FILM, EXTENDED RELEASE;TRANSDERMAL , Discontinued , JANSSEN PHARMS


### 10 bonus points for following the link and getting the strength information from that page.  Print the strength information out along with everything you were getting above.